In [8]:
import queryer
import numpy as np

In [2]:
egdata = queryer.get_earthquake_eager()

execing
execed


In [3]:
egdata

[{'data': [{'stationlat': '53.4064',
    'stationlon': '6.4817',
    'ts': array([  57,   46, -104, ..., -105,   35,  -13])},
   {'stationlat': '53.4064',
    'stationlon': '6.4817',
    'ts': array([ 80, 122, -79, ..., -96, 103,  84])},
   {'stationlat': '53.4064',
    'stationlon': '6.4817',
    'ts': array([-306,  201,  265, ..., -327, -271,  247])},
   {'stationlat': '53.4064',
    'stationlon': '6.4817',
    'ts': array([-135,  -65, -168, ..., -237, -127,  -84])},
   {'stationlat': '52.8944',
    'stationlon': '6.6337',
    'ts': array([-171, -240, -333, ..., -222, -134, -225])},
   {'stationlat': '52.8944',
    'stationlon': '6.6337',
    'ts': array([  17,    7, -105, ...,  -89,   17,  -26])},
   {'stationlat': '52.8944',
    'stationlon': '6.6337',
    'ts': array([ 503,  -76, -865, ..., -111,  489, -259])},
   {'stationlat': '52.8944',
    'stationlon': '6.6337',
    'ts': array([ 133,    5, -367, ..., -179,  145,  -60])},
   {'stationlat': '53.2562',
    'stationlon': '5.5481

In [6]:
print([k for k in egdata[0]])# for one earthquake here are the dictionary keys

['magnitude', 'data', 'eventid', 'eventlat', 'eventlon']


# TODO #
1) Write a function which takes a portion of every time series for an earthquake and returns a single 2d 'image'


In [52]:
def tile_creater(manytimeseries,slicelen =1, metric = np.max):
    """manytimeseries is a list of dictionaries
    like so {'stationlat': , 'stationlon':, 'ts':numpy array}
    we need to calculate the metric for each one, and then place
    each metric in a 2d grid according to its latitude and longitude
    metric is a function that operates on a 2d array, and returns """
    dataarray = np.asarray([i['data'] for i in manytimeseries])#a n rows by t columns array where n is the num sensors
    metrics = slicecalcer(dataarray, slicelen, metric) #now columns are a metric of each slice of the original,
    #and rows still correspond with an individual sensor each. now for each column create a tile. something
    #like
    stationdeets = np.asarray([[i.get('stationlat'),i.get('stationlon')]
                                for i in manytimeseries])
    tiles = [singletile(metrics[:,idx], stationdeets)
             for idx in range(np.shape(metrics)[1])]
    return tiles
def singletile(metricslice, stationdeets):
    """create a tile from a column metricslice which has the metric value for
    #each station for r a given time slice. stationdeets is another numpy array
    which is n rows x 2, where each row is the lat, lon of the station at the corresponding
    row in metricslice:
    Return -> A 2d numpy array tile"""
    pass
def slicecalcer(dataarray, slicelen, metric):
    n,t=np.shape(dataarray)
    slice_steps = np.arange(0,t+1,slicelen)
    slices = [dataarray[:,i[0]:i[1]] for i in zip(slice_steps, slice_steps[1::])]
    metrics = np.asarray([metric(arr, axis=1) for arr in slices]).T
    return metrics
#probably best to create another function which returns grid values which we will use in tile_creater
#assuming we will use a square for now, can always change later
def grid_translation(topleft,size, stationlat, stationlon):
    """Args: 
    topleft is a tuple of the (lat, lon) of the most NW part of our grid
    size is the length in metres of the side of our grid (e.g 100,000)
    stationlat,stationlon are the coordinates of the location we want translated
    Returns:
    (x,y) real valued x,y coordinates where 0<x<size, 0<y<size"""
    pass
def discretiser(size, resolution, x,y):
    """Args: 
    size is the length in metres of the side of our grid (e.g 100,000)
    resolution is the size of the side of one pixel in the grid e.g. 100m
    (x,y) real valued x,y coordinates where 0<x<size, 0<y<size
    Returns:
    (n,m) where n,m are integer valued pixel numbers e.g. (102,88)"""
    pass

In [53]:
a = np.asarray([range(6) for i in range(5)])
print(a)

[[0 1 2 3 4 5]
 [0 1 2 3 4 5]
 [0 1 2 3 4 5]
 [0 1 2 3 4 5]
 [0 1 2 3 4 5]]


In [55]:
slicecalcer(a, slicelen=3, metric=np.max)

array([[2, 5],
       [2, 5],
       [2, 5],
       [2, 5],
       [2, 5]])

In [58]:
[(idx, col) for idx, col in enumerate(a.T)]

[(0, array([0, 0, 0, 0, 0])),
 (1, array([1, 1, 1, 1, 1])),
 (2, array([2, 2, 2, 2, 2])),
 (3, array([3, 3, 3, 3, 3])),
 (4, array([4, 4, 4, 4, 4])),
 (5, array([5, 5, 5, 5, 5]))]